<a href="https://colab.research.google.com/github/monta0315/pytorch_pra/blob/main/buildmodel_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ニューラルネットワークはレイヤー（モジュール）の塊で構成
tourch.nnで構築

In [2]:
%matplotlib inline

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import  datasets,transforms

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


#クラスの定義
nn.moduleを継承し、独自のニューラルネットワークを定義し、その後のネットワークのレイヤーを__init__で初期化する
nn.moduleを継承した全てのモジュールは入力データである順伝播関数であるforword関数を持つ

In [5]:
from torch.nn.modules import linear
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten =  nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


モデルを利用するためには、入力データを与える必要がある
入力データをモデルに投入するとforward関数で処理するとともにいくつかのbackgroundoperationsが実行される
そのため、model.forward()と処理しては行けない

今回のモデルにデータを与えると各クラスの生の予測値を含む10次元のテンソルを返す
nn.softmaxモジュールにこの出力結果を与えることで、入力データが各クラスに属する確率の予測値を求めることができる

In [7]:
X = torch.rand(1,28,28,device=device)
print(X)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[[2.8547e-02, 8.8291e-01, 8.5527e-01, 9.7479e-01, 4.2234e-01,
          2.5545e-02, 8.8373e-01, 8.8922e-01, 4.6560e-01, 5.8928e-01,
          7.1818e-01, 7.2105e-01, 3.8250e-01, 9.3117e-01, 3.4347e-02,
          2.9115e-01, 5.3580e-01, 6.5166e-02, 1.5417e-01, 8.3660e-01,
          8.1849e-01, 3.4793e-01, 6.1489e-01, 2.8478e-01, 8.2009e-01,
          3.0203e-01, 9.7630e-01, 2.8259e-01],
         [7.2598e-01, 4.6867e-01, 9.4671e-01, 4.9512e-01, 4.6459e-01,
          1.6260e-01, 2.7993e-01, 2.2339e-01, 8.1853e-01, 8.9008e-01,
          5.1195e-01, 5.1997e-01, 9.1786e-01, 7.1968e-03, 5.2666e-01,
          4.9971e-01, 7.8524e-03, 1.7741e-01, 3.2455e-01, 5.9594e-01,
          3.3010e-01, 5.3682e-01, 5.7743e-01, 1.8115e-01, 3.3265e-01,
          5.4226e-01, 1.7629e-01, 8.2944e-01],
         [4.9466e-01, 9.6260e-01, 7.8886e-01, 3.1613e-02, 3.1050e-01,
          1.8888e-01, 2.2742e-01, 7.1561e-01, 8.2800e-01, 8.0766e-01,
          7.4444e-01, 8.9336e-01, 6.4822e-01, 2.3441e-01, 5.8614e-

#モデルレイヤー
サイズ28*28の３枚の画像からなるミニバッチのサンプルを用意する

In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
nn.Flattenレイヤーで二次元(28*28)の画像を、1次元の784ピクセルの値へと変換する
ミニバッチの0次元目はサンプル番号を示す次元でこの次元はnn.Flattenを通しても変化しない

In [9]:
#nn.flattenレイヤー
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(input_image.size())
print(flat_image.size())

torch.Size([3, 28, 28])
torch.Size([3, 784])


#nn.Linear
linear_layerは線形変換を施します＝＞inputで前の層のサイズを受け取ってoutputで変換したいラベル数に変換
linear layerは重みとバイアスのパラメータを保持している

In [10]:
layer1 = nn.Linear(in_features=784,out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLu
非線形な活性化関数はニューラルネットワークの入力と出力の間にある複雑な関係性を表現するのに重要な要素
これらは活性化関数の線形変換の後に非線形性を加えニューラルネットワークの表現力を向上させる

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1080,  0.4248, -0.0817, -0.1225,  0.2427, -0.0938, -0.3603,  0.1902,
         -0.2964, -0.1737,  0.6111, -0.3730, -0.2087,  0.2685,  0.0957,  0.1420,
          0.3176,  0.1956,  0.2258,  0.7737],
        [-0.4898,  0.6161,  0.2033,  0.4319,  0.3017, -0.1329,  0.0991, -0.0050,
         -0.5110, -0.5223,  0.2197,  0.0526, -0.1396, -0.0138,  0.0627,  0.1823,
          0.3204,  0.2687,  0.0041,  0.4765],
        [-0.1007,  0.2797,  0.0823,  0.3257,  0.0645, -0.0568,  0.1974,  0.3998,
         -0.2636, -0.3114,  0.2634, -0.3282,  0.0595, -0.0317, -0.0544,  0.1262,
          0.3395,  0.1179,  0.0358,  0.8704]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.1080, 0.4248, 0.0000, 0.0000, 0.2427, 0.0000, 0.0000, 0.1902, 0.0000,
         0.0000, 0.6111, 0.0000, 0.0000, 0.2685, 0.0957, 0.1420, 0.3176, 0.1956,
         0.2258, 0.7737],
        [0.0000, 0.6161, 0.2033, 0.4319, 0.3017, 0.0000, 0.0991, 0.0000, 0.0000,
         0.0000, 0.2197, 0.0526, 0.0000, 0.0000, 0.062

nn.Sequential
モジュールを順番に格納する箱のようなもの
入力データはSequentialに定義された順番に各モジュールを伝搬する

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(2,28,28)
logits = seq_modules(input_image)

nn.softmax
ニューラルネットワークの最後のlinear layerはlogits[-∞,∞]を出力する
このlogitsはnn.Softmaxモジュールへと渡される
その結果、最終的な値は[0~1]の範囲となり、各クラスの確率を表現する
dimパラメータは次元を示していて、dim=1の次元で和を求めると確率の総和なので1になる

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

ニューラルネットワークを構成する多くのモジュールは各々パラメータを保持している
例えば重みやバイアスであり、これらが訓練時に最適化される

nn.Moduleを継承することで、モデルオブジェクト内で定義された全てのフィールドが自動的に追跡できるようになりparameters()やnamed_parameters()メソッドを使って、モデルの各レイヤーの全てのパラメータにアクセスできるようになる
以下ではfor文を用いて各パラメータを処理し、そのサイズと値を表示している

In [17]:
print(model)

for name,param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)
Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0299,  0.0289, -0.0056,  ..., -0.0285, -0.0082, -0.0242],
        [ 0.0232,  0.0313,  0.0202,  ...,  0.0195, -0.0102, -0.0112]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0238, -0.0168], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0007,  0.0238,  0.0376,  ..., -0.0394, -0.0129, -0.0384],
        [ 0.0172,  0.0153,  0.0250,  ...,  0.0421,  0.0052,  0.0032]],
       device='cuda:0', grad_fn=<SliceBa